# 지하철 만족도에 영향을 끼칠 수 있는 변수들 조사

승강장 심도(깊이)

역 면적(크기)

화장실 개수(남+여 = 1개)

화장실 게이트 외부 비율(게이트 외부 화장실/전체 화장실 X 100(%))

In [1]:
import pandas as pd

# 서울시 역 + 자치구 데이터프레임

서울특별시 내의 지하철 역이 어느 자치구에 속해있는지 정리해놓은 데이터틀

In [2]:
subway_df = pd.read_csv('./name_sub_자치구최종.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
subway_df.head()

,호선,정거장명,군구
0,1,가산디지털단지,금천구
1,1,독산,금천구
2,1,금천구청,금천구
3,1,남영,용산구
4,1,용산,용산구


In [3]:
# 호선+정거장명 합쳐서 한 column으로 합치기
subway_df['호선역명'] = subway_df.apply(lambda r:str(r['호선'])+' '+r['정거장명'],axis=1)
subway_df.head()

,호선,정거장명,군구,호선역명
0,1,가산디지털단지,금천구,1 가산디지털단지
1,1,독산,금천구,1 독산
2,1,금천구청,금천구,1 금천구청
3,1,남영,용산구,1 남영
4,1,용산,용산구,1 용산


# 역사 심도 데이터 프레임

지하철 역이 지표면과 얼마나 떨어져 있는지 (+는 지하, -는 지상)

ⓐ지반고(해발고도+100.3m)

ⓑ레일면고(해발고도+100.3m)

정거장 깊이 선로기준 (ⓐ-ⓑ)(m)

정거장 깊이 승강장기준 (ⓐ-ⓑ-1.1)(m)

https://www.data.go.kr/data/15071319/fileData.do

In [4]:
meter_sub = pd.read_csv('./서울교통공사_역사 심도 정보_20220331.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
meter_sub

,연번,호선,정거장명,정거장층수,형식,지반고,레일면고,선로기준 정거장 깊이(m),승강장기준 정거장 깊이(m),비고
0,270,8,산성,B4,상대식,208.65,152.86,55.79,54.69,최대심도역
1,199,6,버티고개,B5,상대식,168.27,122.52,45.75,44.65,최대심도역
2,245,7,숭실대,B6,상대식,171.78,126.15,45.63,44.53,최대심도역
3,149,5,신금호,B8,섬식,159.03,115.77,43.26,42.16,최대심도역
4,180,6,독바위,B6,상대식,138.67,100.73,37.94,36.84,NaN
...,...,...,...,...,...,...,...,...,...,...
270,43,2,대림,고가,상대식,108.80,126.25,-17.45,-18.55,7호선환승
271,24,2,강변,고가,상대식,107.33,125.28,-17.95,-19.05,NaN
272,20,2,뚝섬,고가,상대식,109.75,127.72,-17.97,-19.07,NaN
273,117,4,동작,고가,상대식,105.43,125.23,-19.80,-20.90,9호선환승


필요없는 column 제거 (연번, 정거장 층수, 형식, 비고)

In [5]:
meter_sub = meter_sub.drop(['연번','정거장층수','형식','비고'],axis=1)
meter_sub.head()

,호선,정거장명,지반고,레일면고,선로기준 정거장 깊이(m),승강장기준 정거장 깊이(m)
0,8,산성,208.65,152.86,55.79,54.69
1,6,버티고개,168.27,122.52,45.75,44.65
2,7,숭실대,171.78,126.15,45.63,44.53
3,5,신금호,159.03,115.77,43.26,42.16
4,6,독바위,138.67,100.73,37.94,36.84


호선+정거장명 합쳐서 한 column으로 합치기

In [6]:
meter_sub['호선역명'] = meter_sub.apply(lambda r:str(r['호선'])+' '+r['정거장명'],axis=1)
meter_sub.head()

,호선,정거장명,지반고,레일면고,선로기준 정거장 깊이(m),승강장기준 정거장 깊이(m),호선역명
0,8,산성,208.65,152.86,55.79,54.69,8 산성
1,6,버티고개,168.27,122.52,45.75,44.65,6 버티고개
2,7,숭실대,171.78,126.15,45.63,44.53,7 숭실대
3,5,신금호,159.03,115.77,43.26,42.16,5 신금호
4,6,독바위,138.67,100.73,37.94,36.84,6 독바위


호선역명 column 기준으로 역사 심도, 역 이름 데이터 프레임 병합

In [7]:
meter_merge = pd.merge(meter_sub,subway_df,how = 'left',on='호선역명')
meter_merge.head()

,호선_x,정거장명_x,지반고,레일면고,선로기준 정거장 깊이(m),승강장기준 정거장 깊이(m),호선역명,호선_y,정거장명_y,군구
0,8,산성,208.65,152.86,55.79,54.69,8 산성,NaN,NaN,NaN
1,6,버티고개,168.27,122.52,45.75,44.65,6 버티고개,6,버티고개,중구
2,7,숭실대,171.78,126.15,45.63,44.53,7 숭실대,NaN,NaN,NaN
3,5,신금호,159.03,115.77,43.26,42.16,5 신금호,5,신금호,성동구
4,6,독바위,138.67,100.73,37.94,36.84,6 독바위,6,독바위,은평구


In [8]:
# 불필요한 column 삭제
meter_merge = meter_merge.drop(['호선_x','정거장명_x','호선_y','정거장명_y'],axis=1)
meter_merge.head()

,지반고,레일면고,선로기준 정거장 깊이(m),승강장기준 정거장 깊이(m),호선역명,군구
0,208.65,152.86,55.79,54.69,8 산성,NaN
1,168.27,122.52,45.75,44.65,6 버티고개,중구
2,171.78,126.15,45.63,44.53,7 숭실대,NaN
3,159.03,115.77,43.26,42.16,5 신금호,성동구
4,138.67,100.73,37.94,36.84,6 독바위,은평구


groupby로 자치구 별로 묶어 지하철 역 깊이 평균 계산

In [9]:
meter_GU = meter_merge.groupby('군구').mean().apply(lambda v:round(v,3))
meter_GU

,지반고,레일면고,선로기준 정거장 깊이(m),승강장기준 정거장 깊이(m)
군구,,,,
강남구,125.939,107.752,18.187,17.087
강동구,121.687,103.519,18.168,17.068
강북구,136.745,123.640,13.105,12.005
강서구,114.388,97.417,16.971,15.871
관악구,133.050,119.870,13.180,12.080
광진구,118.305,112.345,5.960,4.860
구로구,116.925,104.323,12.602,11.502
금천구,109.590,81.820,27.770,26.670
노원구,124.766,112.188,12.578,11.478


2021년 서울 서베이 설문조사에서 구한 지하철 이용 만족도 데이터를 활용하여 자치구별 만족도 평균을 구함

In [10]:
aQ8_2 =  pd.read_csv('./2021 자치구별 평균 만족도.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
aQ8_2 = aQ8_2.set_index('GU')
aQ8_2

,AQ8_2
GU,
강남구,3.934746
강동구,3.881182
강북구,3.990856
강서구,3.969712
관악구,4.008000
광진구,3.972955
구로구,4.004695
금천구,3.893539
노원구,3.948116


군구 column 기준으로 데이터 프레임 병합

In [11]:
meter_2021 = pd.merge(meter_GU,aQ8_2,how = 'inner',left_index = True, right_index = True)
meter_2021.head()

,지반고,레일면고,선로기준 정거장 깊이(m),승강장기준 정거장 깊이(m),AQ8_2
군구,,,,,
강남구,125.939,107.752,18.187,17.087,3.934746
강동구,121.687,103.519,18.168,17.068,3.881182
강북구,136.745,123.640,13.105,12.005,3.990856
강서구,114.388,97.417,16.971,15.871,3.969712
관악구,133.050,119.870,13.180,12.080,4.008000


In [12]:
meter_2021.corr(method='pearson')

,지반고,레일면고,선로기준 정거장 깊이(m),승강장기준 정거장 깊이(m),AQ8_2
지반고,1.000000,0.850707,-0.149416,-0.149413,0.269422
레일면고,0.850707,1.000000,-0.646849,-0.646847,0.282626
선로기준 정거장 깊이(m),-0.149416,-0.646849,1.000000,1.000000,-0.140754
승강장기준 정거장 깊이(m),-0.149413,-0.646847,1.000000,1.000000,-0.140764
AQ8_2,0.269422,0.282626,-0.140754,-0.140764,1.000000


2021년도 만족도(AQ8_2) 와의 상관관계

지반고 0.269422

레일면고	0.282626

선로기준 정거장 깊이(m)  -0.140754	

승강장기준 정거장 깊이(m)	  -0.140764	

# 지하철 역사 면적 데이터 프레임

단위는 제곱미터. 

In [13]:
area_df = pd.read_csv('./서울교통공사_역사면적정보_20191231.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
area_df.head()

,호선,역명,대합실,승강장
0,1,서울,8725.00,2080.00
1,1,시청,6863.00,3784.00
2,1,종각,7737.43,2672.81
3,1,종로3가,5628.00,3683.00
4,1,종로5가,7822.00,2208.00


In [14]:
# 호선+역명 합쳐서 한 column으로 합치기
area_df['호선역명'] = area_df.apply(lambda r:str(r['호선'])+' '+r['역명'],axis=1)
area_df.head()

,호선,역명,대합실,승강장,호선역명
0,1,서울,8725.00,2080.00,1 서울
1,1,시청,6863.00,3784.00,1 시청
2,1,종각,7737.43,2672.81,1 종각
3,1,종로3가,5628.00,3683.00,1 종로3가
4,1,종로5가,7822.00,2208.00,1 종로5가


호선역명 column 기준으로 데이터프레임 병합

In [15]:
area_merge = pd.merge(area_df,subway_df,how = 'left',on='호선역명')
area_merge.head()

,호선_x,역명,대합실,승강장,호선역명,호선_y,정거장명,군구
0,1,서울,8725.00,2080.00,1 서울,1,서울,중구
1,1,시청,6863.00,3784.00,1 시청,1,시청,중구
2,1,종각,7737.43,2672.81,1 종각,1,종각,종로구
3,1,종로3가,5628.00,3683.00,1 종로3가,1,종로3가,종로구
4,1,종로5가,7822.00,2208.00,1 종로5가,1,종로5가,종로구


In [16]:
#필요엾는 column 제거
area_merge = area_merge.drop(['호선_x','정거장명','호선_y'],axis=1)
area_merge.head()

,역명,대합실,승강장,호선역명,군구
0,서울,8725.00,2080.00,1 서울,중구
1,시청,6863.00,3784.00,1 시청,중구
2,종각,7737.43,2672.81,1 종각,종로구
3,종로3가,5628.00,3683.00,1 종로3가,종로구
4,종로5가,7822.00,2208.00,1 종로5가,종로구


In [17]:
area_GU = area_merge.groupby('군구').mean().apply(lambda v:round(v,3))
area_GU.head()

,대합실,승강장
군구,,
강남구,6613.813,2490.248
강동구,5426.788,2122.813
강북구,3826.500,2218.500
강서구,4465.551,2246.183
관악구,4027.500,1861.000


2021년 자치구별 지하철 이용 만족도와 병합

In [18]:
area_2021 = pd.merge(area_GU,aQ8_2,how = 'inner',left_index = True, right_index = True)
area_2021.head()

,대합실,승강장,AQ8_2
군구,,,
강남구,6613.813,2490.248,3.934746
강동구,5426.788,2122.813,3.881182
강북구,3826.500,2218.500,3.990856
강서구,4465.551,2246.183,3.969712
관악구,4027.500,1861.000,4.008000


In [19]:
area_2021.corr(method='pearson')

,대합실,승강장,AQ8_2
대합실,1.000000,0.556619,-0.013290
승강장,0.556619,1.000000,0.094735
AQ8_2,-0.013290,0.094735,1.000000


2021년도 만족도(AQ8_2) 와의 상관관계

대합실 면적  -0.013290

승강장 면적  0.094735

# 지하철 화장실 데이터프레임
개수, 역외 역내 비율

1~9호선, 경춘선,분당선,경의중앙선 화장실 정보를 모두 합침

In [20]:
sub_1 = pd.read_csv('./화장실/국가철도공단_수도권1호선_화장실_10_28_2021.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
sub_2 = pd.read_csv('./화장실/국가철도공단_수도권2호선_화장실_10_28_2021.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
sub_3 = pd.read_csv('./화장실/국가철도공단_수도권3호선_화장실_20211118.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
sub_4 = pd.read_csv('./화장실/국가철도공단_수도권4호선_화장실_20211122.csv', sep = ',', header = 0, engine = 'python', encoding='cp949') 
sub_5 = pd.read_csv('./화장실/국가철도공단_수도권5호선_화장실_20211118.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
sub_6 = pd.read_csv('./화장실/국가철도공단_수도권6호선_화장실_20211122.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
sub_7 = pd.read_csv('./화장실/국가철도공단_수도권7호선_화장실_10_28_2021.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
sub_8 = pd.read_csv('./화장실/국가철도공단_수도권8호선_화장실_10_28_2021.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
sub_9 = pd.read_csv('./화장실/국가철도공단_수도권9호선_화장실_10_28_2021.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
sub_GC = pd.read_csv('./화장실/국가철도공단_경춘선_화장실_20211122.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
sub_BD = pd.read_csv('./화장실/국가철도공단_분당선_화장실_20211122.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
sub_KJ = pd.read_csv('./화장실/국가철도공단_경의중앙선_화장실_10_28_2021.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')

In [21]:
sub_Rest = pd.concat([sub_1,sub_2,sub_3,sub_4,sub_5,sub_6,sub_7,sub_8,sub_9,sub_GC,sub_BD,sub_KJ])
sub_Rest.head()

,철도운영기관명,선명,역명,지상지하구분,역층,게이트내외,출구번호,상세위치,남녀구분코드
0,코레일,1호선,소요산,지상,1.0,외,NaN,맞이방,공용
1,코레일,1호선,동두천,지상,3.0,외,1.0,여객통로(1번출구방면),공용
2,코레일,1호선,보산,지상,2.0,외,NaN,맞이방,공용
3,코레일,1호선,보산,지상,1.0,외,1.0,1번 출구,공용
4,코레일,1호선,지행,지상,1.0,외,4.0,4번출구(남부역사),공용


필요한 column(철도운영기관명, 지상지하구분, 역층, 출구번호, 상세위치) 제외

In [22]:
sub_Rest = sub_Rest.drop(['철도운영기관명','지상지하구분','역층','출구번호','상세위치'],axis=1)
sub_Rest.to_csv('./2021화장실개수.csv', index = False, encoding='cp949')
sub_Rest.head(10)

,선명,역명,게이트내외,남녀구분코드
0,1호선,소요산,외,공용
1,1호선,동두천,외,공용
2,1호선,보산,외,공용
3,1호선,보산,외,공용
4,1호선,지행,외,공용
5,1호선,지행,외,공용
6,1호선,덕정,외,공용
7,1호선,양주,외,공용
8,1호선,녹양,외,공용
9,1호선,가능,외,공용


In [23]:
#데이터 병합을 위해 호선 숫자만 남김
sub_Rest['선명'] = sub_Rest['선명'].apply(lambda v:v.replace('호선',''))
sub_Rest.head()

,선명,역명,게이트내외,남녀구분코드
0,1,소요산,외,공용
1,1,동두천,외,공용
2,1,보산,외,공용
3,1,보산,외,공용
4,1,지행,외,공용


In [24]:
#역명에서 괄호제거
sub_Rest = sub_Rest.reset_index(drop=True)

import re
pattern = r'\([^)]*\)'
for i in range(len(sub_Rest)):
    sub_Rest['역명'][i] = re.sub(pattern,'',str(sub_Rest['역명'][i]))

In [25]:
# 호선+정거장명 합쳐서 한 column으로 합치기
sub_Rest['호선역명'] = sub_Rest.apply(lambda r:str(r['선명'])+' '+r['역명'],axis=1)
sub_Rest.head()

,선명,역명,게이트내외,남녀구분코드,호선역명
0,1,소요산,외,공용,1 소요산
1,1,동두천,외,공용,1 동두천
2,1,보산,외,공용,1 보산
3,1,보산,외,공용,1 보산
4,1,지행,외,공용,1 지행


호선역명 기준으로 데이터 병합

In [26]:
Rest_merge = pd.merge(sub_Rest,subway_df,how = 'inner',on='호선역명')
Rest_merge.head()

,선명,역명,게이트내외,남녀구분코드,호선역명,호선,정거장명,군구
0,1,도봉산,외,공용,1 도봉산,1,도봉산,도봉구
1,1,도봉,내,공용,1 도봉,1,도봉,도봉구
2,1,도봉,내,공용,1 도봉,1,도봉,도봉구
3,1,방학,내,공용,1 방학,1,방학,도봉구
4,1,창동,외,여자,1 창동,1,창동,도봉구


groupby로 자치구별 구분 후 게이트 외부 화장실 개수 추합 

In [27]:
Rest_merge['count']=0
Rest_inout = Rest_merge.groupby(['군구','게이트내외'],as_index=False).count().loc[:,['게이트내외','군구','count']]
Rest_inout = Rest_inout.groupby('게이트내외').get_group('외')
Rest_inout = Rest_inout.set_index('군구')
Rest_inout = Rest_inout.drop(['게이트내외'],axis=1)
Rest_inout.columns = ['게이트외']
Rest_inout.head()

,게이트외
군구,
강남구,14
강동구,16
강북구,1
강서구,18
관악구,4


게이트 내부 화장실 개수 추합 

In [28]:
Rest_inout2 = Rest_merge.groupby(['군구','게이트내외'],as_index=False).count().loc[:,['게이트내외','군구','count']]
Rest_inout2 = Rest_inout2.groupby('게이트내외').get_group('내')
Rest_inout2 = Rest_inout2.set_index('군구')
Rest_inout2 = Rest_inout2.drop(['게이트내외'],axis=1)
Rest_inout2.columns = ['게이트내']
Rest_inout2.head()

,게이트내
군구,
강남구,3
강동구,1
강서구,1
관악구,1
광진구,3


자치구별 전체 화장실 개수 총합

In [29]:
Rest_all = Rest_merge.groupby(['군구']).count().loc[:,['역명']]
Rest_all.columns = ['총화장실수']
Rest_all.head()

,총화장실수
군구,
강남구,17
강동구,17
강북구,1
강서구,19
관악구,5


In [30]:
Rest_sum = pd.merge(Rest_all,Rest_inout,how='left',left_index=True,right_index=True)
Rest_sum = pd.merge(Rest_sum,Rest_inout2,how='left',left_index=True,right_index=True)
Rest_sum.head()

,총화장실수,게이트외,게이트내
군구,,,
강남구,17,14,3.0
강동구,17,16,1.0
강북구,1,1,NaN
강서구,19,18,1.0
관악구,5,4,1.0


게이트 외부 화장실/전체 화장실 X 100(%)로 새로운 column 생성하여 데이터처리

In [31]:
Rest_sum['게이트외부화장실비율'] = Rest_sum.apply(lambda v: round(int(v['게이트외'])/int(v['총화장실수'])*100,3),axis=1)
Rest_sum.head()

,총화장실수,게이트외,게이트내,게이트외부화장실비율
군구,,,,
강남구,17,14,3.0,82.353
강동구,17,16,1.0,94.118
강북구,1,1,NaN,100.000
강서구,19,18,1.0,94.737
관악구,5,4,1.0,80.000


2021 지하철 만족도 결합 후 상관관계 분석

In [32]:
Rest_2021 = pd.merge(Rest_sum,aQ8_2,how = 'inner',left_index = True, right_index = True)
Rest_2021 = Rest_2021.fillna(0)
Rest_2021.head()

,총화장실수,게이트외,게이트내,게이트외부화장실비율,AQ8_2
군구,,,,,
강남구,17,14,3.0,82.353,3.934746
강동구,17,16,1.0,94.118,3.881182
강북구,1,1,0.0,100.000,3.990856
강서구,19,18,1.0,94.737,3.969712
관악구,5,4,1.0,80.000,4.008000


In [33]:
Rest_2021.corr(method='pearson')

,총화장실수,게이트외,게이트내,게이트외부화장실비율,AQ8_2
총화장실수,1.000000,0.915646,0.639092,-0.177243,0.226413
게이트외,0.915646,1.000000,0.276002,0.192528,0.285142
게이트내,0.639092,0.276002,1.000000,-0.792160,-0.004212
게이트외부화장실비율,-0.177243,0.192528,-0.792160,1.000000,0.170218
AQ8_2,0.226413,0.285142,-0.004212,0.170218,1.000000


2021년도 만족도(AQ8_2) 와의 상관관계

총화장실수 0.226413

게이트외부 화장실 수  0.285142

게이트내부 화장실 수  -0.004212

게이트외부화장실비율  0.170218

In [34]:
sub_aLL = pd.concat([meter_GU,area_GU,Rest_sum,aQ8_2],axis=1)
sub_aLL = sub_aLL.fillna(0)
sub_aLL

,지반고,레일면고,선로기준 정거장 깊이(m),승강장기준 정거장 깊이(m),대합실,승강장,총화장실수,게이트외,게이트내,게이트외부화장실비율,AQ8_2
강남구,125.939,107.752,18.187,17.087,6613.813,2490.248,17,14,3.0,82.353,3.934746
강동구,121.687,103.519,18.168,17.068,5426.788,2122.813,17,16,1.0,94.118,3.881182
강북구,136.745,123.640,13.105,12.005,3826.500,2218.500,1,1,0.0,100.000,3.990856
강서구,114.388,97.417,16.971,15.871,4465.551,2246.183,19,18,1.0,94.737,3.969712
관악구,133.050,119.870,13.180,12.080,4027.500,1861.000,5,4,1.0,80.000,4.008000
광진구,118.305,112.345,5.960,4.860,4739.852,2224.582,11,8,3.0,72.727,3.972955
구로구,116.925,104.323,12.602,11.502,5754.747,2764.523,14,5,9.0,35.714,4.004695
금천구,109.590,81.820,27.770,26.670,10116.830,2434.700,3,3,0.0,100.000,3.893539
노원구,124.766,112.188,12.578,11.478,6947.807,2484.462,18,13,5.0,72.222,3.948116
도봉구,130.597,130.823,-0.227,-1.327,4253.760,2134.900,8,4,4.0,50.000,3.899433


In [35]:
sub_aLL.to_csv('./2021 승강장심도평균 역사면적평균 화장실개수비율평균 만족도 통계.csv', index = True, encoding='cp949')